In [1]:
# ==============================
# Monte Carlo Option Pricing Simulator
# Kaggle-Ready Animated Version with GIF
# ==============================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.stats import norm
from IPython.display import Image
from matplotlib.animation import PillowWriter

# ------------------------------
# Dark Professional Theme
# ------------------------------
plt.style.use('dark_background')
plt.rcParams['figure.facecolor'] = '#111111'
plt.rcParams['axes.facecolor'] = '#111111'
plt.rcParams['axes.edgecolor'] = 'white'
plt.rcParams['axes.labelcolor'] = 'white'
plt.rcParams['xtick.color'] = 'white'
plt.rcParams['ytick.color'] = 'white'

# ------------------------------
# Parameters
# ------------------------------
S0 = 100
K = 100
r = 0.05
sigma = 0.2
T = 1.0
steps = 120      # Number of time steps
n_paths = 5000   # Total GBM paths (for pricing accuracy)

dt = T / steps

# ------------------------------
# Simulate GBM Paths
# ------------------------------
np.random.seed(42)
Z = np.random.standard_normal((n_paths, steps))
paths = np.zeros((n_paths, steps + 1))
paths[:, 0] = S0

for t in range(1, steps + 1):
    paths[:, t] = paths[:, t-1] * np.exp(
        (r - 0.5*sigma**2)*dt + sigma*np.sqrt(dt)*Z[:, t-1]
    )

time_grid = np.linspace(0, T, steps + 1)

# ------------------------------
# Monte Carlo Pricing
# ------------------------------
call_payoff = np.maximum(paths[:, -1] - K, 0)
put_payoff = np.maximum(K - paths[:, -1], 0)

mc_call = np.exp(-r*T) * np.mean(call_payoff)
mc_put = np.exp(-r*T) * np.mean(put_payoff)

# ------------------------------
# Black-Scholes Pricing
# ------------------------------
d1 = (np.log(S0/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
d2 = d1 - sigma*np.sqrt(T)

bs_call = S0*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
bs_put = K*np.exp(-r*T)*norm.cdf(-d2) - S0*norm.cdf(-d1)

diff_call = mc_call - bs_call
diff_put = mc_put - bs_put

# ------------------------------
# Setup Figure for Animation
# ------------------------------
fig, ax = plt.subplots(figsize=(10,6))
ax.set_title("Monte Carlo GBM Simulation", fontsize=16)
ax.set_xlabel("Time (Years)")
ax.set_ylabel("Stock Price")
ax.set_xlim(0, T)
ax.set_ylim(0.8*S0, 1.5*S0)

# Strike price line
ax.axhline(K, color='red', linestyle='--', linewidth=1.5, label='Strike Price')
ax.legend()

# Display parameters
params_text = f"S0={S0}\nK={K}\nr={r}\nσ={sigma}\nT={T}"
ax.text(0.02, 0.95, params_text,
        transform=ax.transAxes,
        fontsize=11,
        verticalalignment='top',
        bbox=dict(facecolor='#222222', alpha=0.8))

# Dynamic price box
price_box = ax.text(0.98, 0.95, "",
                    transform=ax.transAxes,
                    fontsize=11,
                    verticalalignment='top',
                    horizontalalignment='right',
                    bbox=dict(facecolor='#222222', alpha=0.8))

# Animate only first 10 paths for smooth GIF
lines = []
colors = plt.cm.viridis(np.linspace(0.3, 1, 10))  # gradient coloring
for c in colors:
    line, = ax.plot([], [], lw=2, alpha=0.8, color=c)
    lines.append(line)

# ------------------------------
# Animation Function
# ------------------------------
def animate(i):
    for j, line in enumerate(lines):
        line.set_data(time_grid[:i], paths[j, :i])
    price_box.set_text(
        f"Monte Carlo Call: {mc_call:.4f}\n"
        f"Black-Scholes Call: {bs_call:.4f}\n"
        f"Diff: {diff_call:.4f}\n\n"
        f"Monte Carlo Put: {mc_put:.4f}\n"
        f"Black-Scholes Put: {bs_put:.4f}\n"
        f"Diff: {diff_put:.4f}"
    )
    return lines + [price_box]

# ------------------------------
# Create and Save GIF Animation
# ------------------------------
ani = animation.FuncAnimation(
    fig, animate,
    frames=steps+1,
    interval=50,
    blit=True
)

plt.close(fig)
ani.save("/kaggle/working/monte_carlo_sim.gif", writer=PillowWriter(fps=20))

# Display GIF
Image("/kaggle/working/monte_carlo_sim.gif")

# ------------------------------
# Print Price Table
# ------------------------------
df = pd.DataFrame({
    "Method": ["Monte Carlo", "Black-Scholes"],
    "Call Price": [mc_call, bs_call],
    "Put Price": [mc_put, bs_put]
})

print("\nOption Price Comparison:")
print(df.to_string(index=False))
print(f"\nCall Difference (MC - BS): {diff_call:.4f}")
print(f"Put Difference (MC - BS): {diff_put:.4f}")

# ------------------------------
# Put-Call Parity Check
# ------------------------------
lhs = mc_call - mc_put
rhs = S0 - K*np.exp(-r*T)

print("\nPut-Call Parity Check:")
print(f"LHS (C - P): {lhs:.4f}")
print(f"RHS (S0 - Ke^-rT): {rhs:.4f}")



Option Price Comparison:
       Method  Call Price  Put Price
  Monte Carlo   10.264582   5.685307
Black-Scholes   10.450584   5.573526

Call Difference (MC - BS): -0.1860
Put Difference (MC - BS): 0.1118

Put-Call Parity Check:
LHS (C - P): 4.5793
RHS (S0 - Ke^-rT): 4.8771
